In [1]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

print("CUDA Available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Device Name:", torch.cuda.get_device_name(0))

device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device)

CUDA Available: True
Device Name: NVIDIA GeForce RTX 4070 Laptop GPU
cuda:0


In [2]:


torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3-turbo"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
)

result = pipe("./Recording.mp3")
print(result["text"])

c:\Users\utsav\OneDrive\Desktop\Apeksha\env\Lib\site-packages\transformers\models\whisper\generation_whisper.py:509: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


 Hey, so this is test2 for testing my 10 second audio stream. Whereas 5 seconds has been already passed and I'll be stopping at the 10th. Thank you.


In [3]:
import ollama
def get_response(prompt):
    response = ollama.chat(model='llama3.2', 
                           messages=[{'role': 'user', 'content': prompt}])
    return response['message']['content']

get_response("What is the country known for having the best weather in the world?")

'It\'s difficult to pinpoint a single country as having the "best" weather in the world, as different people may have varying preferences when it comes to climate and weather conditions. However, according to various surveys, reports, and studies, some of the countries that are often considered to have the best weather include:\n\n1. **Hawaii (USA)**: Known for its tropical climate, Hawaii is often referred to as the "most perfect" place on Earth. Its average temperature ranges from 70°F to 85°F (21°C to 30°C) throughout the year.\n2. **Barbados**: This Caribbean island nation enjoys a subtropical climate with mild temperatures, averaging around 77°F (25°C) and low humidity.\n3. **Maldives**: Located in the Indian Ocean, Maldives has a tropical monsoon climate with warm temperatures and low humidity.\n4. **New Zealand**: Known for its temperate climate, New Zealand\'s weather is generally mild, with average temperatures ranging from 50°F to 70°F (10°C to 21°C) throughout the year.\n5. 

In [4]:
import pyaudio
import wave

def record_audio(filename="prompt.mp3", duration=3, sample_rate=44100, channels=1, chunk=1024):
    p = pyaudio.PyAudio()

    stream = p.open(format=pyaudio.paInt16,
                    channels=channels,
                    rate=sample_rate,
                    input=True,
                    frames_per_buffer=chunk)

    print("Recording...")

    frames = []

    for i in range(0, int(sample_rate / chunk * duration)):
        data = stream.read(chunk)
        frames.append(data)

    print("Recording finished.")

    stream.stop_stream()
    stream.close()
    p.terminate()

    # Save the recorded data as a WAV file
    wf = wave.open(filename.replace('.mp3', '.wav'), 'wb')
    wf.setnchannels(channels)
    wf.setsampwidth(p.get_sample_size(pyaudio.paInt16))
    wf.setframerate(sample_rate)
    wf.writeframes(b''.join(frames))
    wf.close()

    print(f"Audio saved as {filename.replace('.mp3', '.wav')}")

# record_audio()

In [5]:
def transcribe(audio_filepath):
    result = pipe(audio_filepath)
    return result["text"]

transcribe("./prompt.wav")

' hey is this file recording itself'

In [7]:
record_audio()
prompt = transcribe("./prompt.wav")
print(f"Question: {prompt}")
get_response(prompt)

Recording...
Recording finished.
Audio saved as prompt.wav


c:\Users\utsav\OneDrive\Desktop\Apeksha\env\Lib\site-packages\transformers\models\whisper\generation_whisper.py:509: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


Question:  What is artificial intelligence?


'Artificial intelligence (AI) refers to the development of computer systems that can perform tasks that typically require human intelligence, such as:\n\n1. Learning: AI systems can learn from data and improve their performance over time.\n2. Problem-solving: AI systems can analyze problems and find solutions.\n3. Reasoning: AI systems can draw conclusions based on available information.\n4. Perception: AI systems can interpret and understand data from sensors, such as images and speech.\n\nAI has many applications in various fields, including:\n\n1. Virtual assistants: AI-powered virtual assistants, like Siri, Alexa, and Google Assistant, can perform tasks, answer questions, and provide information.\n2. Image recognition: AI systems can recognize objects, faces, and patterns in images.\n3. Natural language processing (NLP): AI systems can understand, interpret, and generate human language.\n4. Robotics: AI systems can control robots to perform tasks, such as assembly, packaging, and l

In [8]:
# Creating the task db first before writing the tools for the model
import pandas as pd
from datetime import datetime

# Create an empty DataFrame for the tasks database
tasks_df = pd.DataFrame(columns=['task', 'status', 'creation_date', 'completed_date'])

tasks_df

,task,status,creation_date,completed_date


In [9]:
def add_task(task_description):
    """
    Add a task to the tasks database.
    """
    new_task = pd.DataFrame({
        'task': [task_description],
        'status': ['Not Started'],
        'creation_date': [datetime.now().strftime('%Y-%m-%d %H:%M:%S')],
        'completed_date': [None]
    })
    global tasks_df
    tasks_df = pd.concat([tasks_df, new_task], ignore_index=True)
    
    return tasks_df

In [10]:
tool_add_tasks_to_db = {
    'type': 'function',
    'function': {
        'name': 'add_task',
        'description': 'Add a task to the tasks database',
        'parameters': {
            'type': 'object',
            'properties': {
                'task_description': {
                    'type': 'string',
                    'description': 'The description of the task to add',
                },
            },
            'required': ['task_description'],
        },
    },
}

In [11]:
# Creating tasks in a backlog task db
def get_response_with_tools(prompt):
    response = ollama.chat(model='llama3.2', 
                           messages=[{'role': 'user', 'content': prompt}],
                           tools=[tool_add_tasks_to_db])
    # Process tool calls if present
    if 'tool_calls' in response['message']:
        for tool_call in response['message']['tool_calls']:
            if tool_call['function']['name'] == 'add_task':
                task_description = tool_call['function']['arguments']['task_description']
                add_task(task_description)
                print(f"Task added: {task_description}")
    else:
        return response['message']['content']

In [14]:
get_response_with_tools("Create a task to create a local voice AI assistant")
tasks_df

Task added: create a local voice AI assistant


,task,status,creation_date,completed_date
0,create a local voice AI assistant,Not Started,2024-12-01 14:58:50,None
1,create a local voice AI assistant,Not Started,2024-12-01 15:00:11,None


In [15]:
tool_create_file = {
            'type': 'function',
            'function': {
                'name': 'create_file',
                'description': 'Create a new file with given content',
                'parameters': {
                    'type': 'object',
                    'properties': {
                        'filename': {
                            'type': 'string',
                            'description': 'The name of the file to create',
                        },
                        'content': {
                            'type': 'string',
                            'description': 'The content to write to the file',
                        },
                    },
                    'required': ['filename', 'content'],
                },
            },
        }
tool_read_file = {
            'type': 'function',
            'function': {
                'name': 'read_file',
                'description': 'Read the content of a file',
                'parameters': {
                    'type': 'object',
                    'properties': {
                        'filename': {
                            'type': 'string',
                            'description': 'The name of the file to read',
                        },
                    },
                    'required': ['filename'],
                },
            },
        }
tool_delete_file = {
            'type': 'function',
            'function': {
                'name': 'delete_file',
                'description': 'Delete a file',
                'parameters': {
                    'type': 'object',
                    'properties': {
                        'filename': {
                            'type': 'string',
                            'description': 'The name of the file to delete',
                        },
                    },
                    'required': ['filename'],
                },
            },
        }

tool_edit_file = {
            'type': 'function', 
            'function': {
                'name': 'edit_file',
                'description': 'Edit the content of a file',
                'parameters': {
                    'type': 'object',
                    'properties': {
                        'filename': {
                            'type': 'string',
                            'description': 'The name of the file to edit',
                        },
                        'content': {
                            'type': 'string',
                            'description': 'The content to write to the file',
                        },
                    },
                    'required': ['filename', 'content'],
                },
            },
        }
tools = [tool_create_file, tool_read_file, tool_delete_file, tool_add_tasks_to_db]

In [16]:
import os
# Writing functions to create, read, edit and delete files

def create_file(filename, content):
    with open(filename, 'w') as file:
        file.write(content)
    return f"File {filename} created successfully"

def read_file(filename):
    with open(filename, 'r') as file:
        return file.read()

def edit_file(filename, content):
    with open(filename, 'w') as file:
        file.write(content)
    return f"File {filename} edited successfully"

def delete_file(filename):
    os.remove(filename)
    return f"File {filename} deleted successfully"

# Creating tasks in a backlog task db
def get_response_with_tools(prompt):
    response = ollama.chat(model='llama3.2', 
                           messages=[{'role': 'user', 'content': prompt}],
                           tools=tools)
    # Process tool calls if present
    if 'tool_calls' in response['message']:
        for tool_call in response['message']['tool_calls']:
            if tool_call['function']['name'] == 'add_task':
                task_description = tool_call['function']['arguments']['task_description']
                add_task(task_description)
                print(f"Task added: {task_description}")
            elif tool_call['function']['name'] == 'create_file':
                print("Creating file...")
                filename = tool_call['function']['arguments']['filename']
                content = tool_call['function']['arguments']['content']
                create_file(filename, content)
                print(f"File created: {filename}")
            elif tool_call['function']['name'] == 'read_file':
                print("Reading file...")
                filename = tool_call['function']['arguments']['filename']
                content = read_file(filename)
                print(f"File content: {content}")
            elif tool_call['function']['name'] == 'delete_file':
                print("Deleting file...")
                filename = tool_call['function']['arguments']['filename']
                delete_file(filename)
                print(f"File deleted: {filename}")
    else:
        return response['message']['content']

In [17]:
get_response_with_tools("Create a file called 'test.txt' with the content 'Hello, world!'")

Creating file...
File created: test.txt
